# Exercise 1: Error metrics

In this notebook we set out exercises to implement error metrics and use them in skforecast. The solutions we show are only one way of answering these questions.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Data loading and preparation

Data has been obtained from [Store Item Demand Forecasting Challenge](https://www.kaggle.com/competitions/demand-forecasting-kernels-only/data), specifically `train.csv`. This dataset contains 913,000 sales transactions from 2013–01–01 to 2017–12–31 for 50 products (SKU) in 10 stores. The goal is to predict the next 7 days sales for 50 different items in one store using the available 5 years history.

In [ ]:
from skforecast.datasets import fetch_dataset

# Load the data
data = fetch_dataset(name="store_sales", raw=True)
data.head()

`ForecasterRecursiveMultiSeries` requires that each time series is a column in the dataframe and that the index is time-like (datetime or timestamp). 

So now we process the data to get dataframes in the required format.

In [ ]:
# Data preprocessing

selected_store = 2
selected_items = data.item.unique()  # All items
# selected_items = [1, 2, 3, 4, 5] # Selection of items to reduce computation time

# Filter data to specific stores and products
mask = (data["store"] == selected_store) & (data["item"].isin(selected_items))
data = data[mask].copy()

# Convert `date` column to datetime
data["date"] = pd.to_datetime(data["date"], format="%Y-%m-%d")

# Convert to one column per time series
data = pd.pivot_table(data=data, values="sales", index="date", columns="item")

# Reset column names
data.columns.name = None
data.columns = [f"item_{col}" for col in data.columns]

# Explicitly set the frequency of the data to daily.
# This would introduce missing values for missing days.
data = data.asfreq("1D")

# Sort by time
data = data.sort_index()

data.head(4)

In [ ]:
# Check if any missing values introduced
data.isnull().sum().any()

In [ ]:
# Plot a subset of the time series
fig, ax = plt.subplots(figsize=(8, 5))
data.iloc[:, :4].plot(
    legend=True,
    subplots=True,
    sharex=True,
    title="Sales of store 2",
    ax=ax,
)
fig.tight_layout()

Let's add the day of the week to use as an exogenous feature.

In [ ]:
data["day_of_week"] = data.index.weekday

# Analysis

We want to build an intuition about the characteristics (trend, outliers, seasonality, intermittency, etc.) and the range of values that the time series have.
This helps when deciding which error metrics to use.

Compute a set of summary statistics and/or plots to do this.

Plot the 5 largest and 5 smallest time series.

Check for any zero values.

# Implementing custom error metrics

Error metrics that we can consider using are:
- To measure the bias: 
    - Average forecast bias: we compute the forecast bias, which is a scale independent measure, for each time series and then average over all time series.
- To measure the error:
    - Normalised RMSE or normalised deviation: these are pooled error metrics, which cannot currently be calculated natively in skforecast.
    - Average RMSSE or MASE:  we can compute the MASE or RMSSE, a scale independent metric, for each time series individually and then average the results to give our final metric. This cannot currently be calculated in skforecast as it requires the custom error function to have
    access to the training set. 
    - Average WAPE: we can compute the WAPE, a scale independent metric, for each time series individually and then average the results to give our final metric.

Create a function that takes `y_true` and `y_pred` as an argument and ouputs the 
weighted mean absolute percentage error (WAPE) metric defined as:

$ WAPE = \frac{\sum_t{|e_t|}}{\sum_t{|y_t|}} $

Note: The data does have trend, this would suggest we should not use WAPE. However,
as we are only forecasting 7 days into the future the magnitude of the trend in
the forecast horizon is negligible. 

Create a function that takes `y_true` and `y_pred` as an argument and ouputs the 
forecast bias metric defined as the mean error divided by the mean of the time series
in the forecast horizon:

$ FB = \frac{1/H\sum_t{e_t}}{1/H\sum_t{y_t}} = \frac{\sum_t{e_t}}{\sum_t{y_t}}$

# Using a custom error metric

Define a model to do recursive multistep forecasting for multiple independent time series.

Perform backtesting with refitting at every backtest step and compute the forecast bias
and WAPE for each time series.

In [ ]:
from skforecast.model_selection import backtesting_forecaster_multiseries

Average the forecast bias and WAPE over all items to get a single value for each
metric. Does your model over or under forecast?

Plot a sample of the largest and smallest WAPE time series and their backtest
predictions.